# Pandas

# Target de todas las combinaciones producto-cliente en todos los periodos siempre que se haya dado la interaccion producto-cliente

In [ ]:
# import pandas as pd
# import gc

# # Cargar datos
# sellin = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/sell-in.txt.gz", sep="\t")
# productos = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_productos.txt", sep="\t")
# stocks = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_stocks.txt", sep="\t")

# df = pd.merge(sellin, productos, how="left", on="product_id")
# df = df.merge(stocks, how="left", on=["product_id", "periodo"])
# del sellin, productos, stocks

# # Agrupar ventas por periodo, cliente y producto
# dt = df.groupby(by=["periodo","customer_id","product_id"]).agg({"tn":"sum",
#                                                                 # "cust_request_tn":"sum",
#                                                                 # "cust_request_qty":"sum",
#                                                                 # "cat1":"first",
#                                                                 # "cat2":"first",
#                                                                 # "cat3":"first",
#                                                                 # "brand":"first",
#                                                                 # "sku_size":"first",
#                                                                 # "stock_final":"first",
#                                                                 # "plan_precios_cuidados":"first"
#                                                                 }).reset_index()


# # # rellenar con ceros
# todos_los_periodos = df['periodo'].unique()

# # 2. Combinaciones reales de producto y cliente
# combinaciones_existentes = df[['product_id', 'customer_id']].drop_duplicates()

# completo = (
#     combinaciones_existentes
#     .assign(key=1)
#     .merge(pd.DataFrame({'periodo': todos_los_periodos, 'key': 1}), on='key')
#     .drop('key', axis=1)
# )

# # 4. Merge con los datos reales
# df_completo = (
#     completo
#     .merge(df, on=['periodo', 'product_id', 'customer_id'], how='left')
# )
# # 4. Rellenar NaNs con 0 en la columna de ventas
# df_completo['tn'] = df_completo['tn'].fillna(0)

(13479804, 17)

# Target de todas las combinaciones producto-cliente desde el periodo en que se dio la primer interaccion producto-cliente

In [1]:
import pandas as pd
import gc

# Cargar datos
sellin = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/sell-in.txt.gz", sep="\t")
productos = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_productos.txt", sep="\t")
stocks = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_stocks.txt", sep="\t")

df = pd.merge(sellin, productos, how="left", on="product_id")
df = df.merge(stocks, how="left", on=["product_id", "periodo"])
del sellin, productos, stocks

# Agrupar ventas por periodo, cliente y producto
dt = df.groupby(by=["periodo","customer_id","product_id"]).agg({"tn":"sum",
                                                                "cust_request_tn":"sum",
                                                                "cust_request_qty":"sum",
                                                                "cat1":"first",
                                                                "cat2":"first",
                                                                "cat3":"first",
                                                                "brand":"first",
                                                                "sku_size":"first",
                                                                "stock_final":"first",
                                                                "plan_precios_cuidados":"first"
                                                                }).reset_index()
# 1. Calcular primeros períodos
primer_interaccion = df.groupby(['product_id', 'customer_id'])['periodo'].min().rename('primer_periodo')

# 2. Crear todas las combinaciones posibles
productos = df['product_id'].unique()
clientes = df['customer_id'].unique()
periodos = df['periodo'].unique()

# 3. Crear matriz de combinaciones válidas usando broadcasting
idx = pd.MultiIndex.from_product([productos, clientes, periodos], names=['product_id', 'customer_id', 'periodo'])
completo = idx.to_frame(index=False)

# 4. Unir con primeros períodos y filtrar
completo = completo.merge(primer_interaccion, on=['product_id', 'customer_id'], how='left')
completo = completo[completo['periodo'] >= completo['primer_periodo']].drop('primer_periodo', axis=1)

# 5. Unir con datos reales
df_completo = completo.merge(df, how='left')
df_completo['tn'] = df_completo['tn'].fillna(0)

# Convertir periodo a datetime con día fijo
df_completo['periodo_dt'] = pd.to_datetime(df_completo['periodo'], format='%Y%m')

# Crear columna periodo_target_dt desplazada 2 meses
df_completo['periodo_target_dt'] = df_completo['periodo_dt'] + pd.DateOffset(months=2)

# DataFrame auxiliar con ventas futuras
ventas_futuras = df_completo[['periodo_dt', 'customer_id', 'product_id', 'tn']].copy()
ventas_futuras = ventas_futuras.rename(columns={
    'periodo_dt': 'periodo_target_dt',
    'tn': 'target'
})

# Hacemos el merge usando fechas datetime directamente
dt = pd.merge(
    df_completo,
    ventas_futuras,
    how='left',
    on=['periodo_target_dt', 'customer_id', 'product_id']
)
dt = dt.drop(columns=['periodo_target_dt']) # Ya usamos esta para hacer el merge
del ventas_futuras, df_completo, completo, primer_interaccion
gc.collect()

dt.shape

(10096720, 16)

In [ ]:
#Prueba target
dt[
    (dt['customer_id'] == 10010) &
    (dt['product_id'] == 20001)
][["periodo","tn","target"]].head(5)

,periodo,tn,target
4537728,201702,16.62160,22.74418
4537729,201703,20.31530,11.73026
4537730,201704,22.74418,11.69668
4537731,201705,11.73026,33.85883
4537732,201706,11.69668,9.84984
4537733,201707,33.85883,28.93391
4537734,201708,9.84984,0.00000
4537735,201709,28.93391,51.30871
4537736,201710,0.00000,0.11193
4537737,201711,51.30871,33.19844


In [2]:

# crear alguans variables nuevas de prueba
dt["month"] = dt["periodo_dt"].dt.month
dt["year"] = dt["periodo_dt"].dt.year


# Convertir columnas categóricas a tipo 'category' para LightGBM
for col in ['cat1', 'cat2', 'cat3', 'brand', 'plan_precios_cuidados','descripcion']:
    if col in dt.columns:
        dt[col] = dt[col].astype('category')

dt_kgl = dt[dt["periodo"].isin([201912])]
dt = dt.drop(dt[dt["periodo"].isin([201911,201912])].index,axis=0)

In [3]:
dt.shape

(9349922, 18)

In [ ]:
dt.columns

Index(['product_id', 'customer_id', 'periodo', 'plan_precios_cuidados',
       'cust_request_qty', 'cust_request_tn', 'tn', 'cat1', 'cat2', 'cat3',
       'brand', 'sku_size', 'descripcion', 'stock_final', 'periodo_dt',
       'periodo_target_dt', 'target'],
      dtype='object')

In [ ]:
# prompt: ajustar un modelo lightgbm de regresion, teniendo como objetivo la columna target

import lightgbm as lgb
from sklearn.model_selection import train_test_split
import numpy as np



# Columnas a utilizar como features (X) y la columna target (y)
# Se excluyen las columnas de identificación y las que no son numéricas o son el target
feature_columns = [col for col in dt.columns if col not in ['periodo','periodo_dt', 'target']]
X = dt[feature_columns]
y = dt['target']



# Dividir los datos en conjuntos de entrenamiento y prueba (opcional, pero recomendado)
# Para este caso, vamos a entrenar con todos los datos disponibles antes de 201911
# Si quisieras validación, podrías usar un periodo anterior como validación

# Definir el modelo LightGBM
lgb_reg = lgb.LGBMRegressor(random_state=12345)

# Entrenar el modelo
lgb_reg.fit(X, y)

print("Modelo LightGBM entrenado con éxito.")

# Ahora puedes usar lgb_reg para hacer predicciones
# Por ejemplo, si tuvieras un dataframe 'X_pred' con los mismos features:
# predictions = lgb_reg.predict(X_pred)



[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.


In [ ]:
# prompt: calcular accuracy del modelo entrenado

import numpy as np
from sklearn.metrics import accuracy_score

# Para calcular la precisión (accuracy), necesitas tener etiquetas binarias (0 o 1).
# Tu target 'tn' es una variable continua representando ventas.
# Accuracy es adecuada para problemas de clasificación.

# Si quieres evaluar un modelo de regresión como LGBMRegressor,
# deberías usar métricas de regresión como Mean Absolute Error (MAE),
# Mean Squared Error (MSE), R-squared, etc.

# Si deseas adaptar esto a un problema de clasificación (ej: predecir si hay venta o no > 0),
# puedes binarizar el target:
# y_binary = (y > 0).astype(int)
# Si ya has entrenado el modelo con el target continuo, no puedes calcular accuracy directamente.

# Asumiendo que quieres evaluar cómo se desempeña el modelo en los datos de entrenamiento
# (lo cual no es una métrica de rendimiento generalizable, pero ilustra el proceso):

# Hacer predicciones en el conjunto de entrenamiento
y_pred = lgb_reg.predict(X)

# Para un problema de regresión, puedes calcular MAE o MSE
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# Si binarizaras el problema a Clasificación (¿Hay venta o no?):
# y_binary = (y > 0).astype(int)
# # Entrenar un clasificador (si es necesario, si el modelo no soporta clasificación binaria)
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(random_state=42)
# clf.fit(X, y_binary)
# y_pred_binary = clf.predict(X)
# accuracy = accuracy_score(y_binary, y_pred_binary)
# print(f"Accuracy (binary classification): {accuracy:.4f}")

# Nota: Evaluar en el mismo conjunto de entrenamiento no da una idea realista del rendimiento
# en datos no vistos. Lo ideal sería dividir los datos en entrenamiento y prueba
# ANTES de entrenar y evaluar en el conjunto de prueba.
# O usar validación cruzada.

Mean Absolute Error (MAE): 0.1174
Mean Squared Error (MSE): 1.0259
Root Mean Squared Error (RMSE): 1.0129


In [ ]:


# Columnas a utilizar como features (X) y la columna target (y)
# Se excluyen las columnas de identificación y las que no son numéricas o son el target
X_kgl = dt_kgl[feature_columns]
y_pred = lgb_reg.predict(X_kgl)

In [ ]:
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
result = pd.DataFrame({"product_id": X_kgl["product_id"],  "tn": y_pred})
result = result[result["product_id"].isin(productos_ok["product_id"])]
result = result.groupby("product_id").agg({"tn":"sum"}).reset_index()
result

,product_id,tn
0,20001,1445.987290
1,20002,971.359486
2,20003,772.756700
3,20004,516.712323
4,20005,478.195130
...,...,...
775,21263,0.382548
776,21265,0.712950
777,21266,0.712950
778,21267,0.088928


In [ ]:
# prompt: descargar en csv el dataframe result

from google.colab import files
result.to_csv('result.csv', index=False,sep=",")
files.download('result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>